## Importing Relevant Libraries

In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

## Data Retrieval

### Setting up Connection to BigQuery

In [2]:
client = bigquery.Client()

### Loading Data from BigQuery

In [3]:
# Define the query to retrieve data from the activities table
activities_query = """
    SELECT * FROM `strava-etl.strava_data.activities`
"""
activities_query_job = client.query(activities_query)

# Convert activities data to a dataframe
activities = activities_query_job.to_dataframe()
activities.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,...,similar_activities_average_speed,similar_activities_min_average_speed,similar_activities_mid_average_speed,similar_activities_max_average_speed,similar_activities_pr_rank,similar_activities_frequency_milestone,elevation_change,day_of_week,hour,month
0,3,2024 Erie Marathon,42346.6,9454,9454,101.0,Run,Run,1,12359295567,...,NaN,NaN,NaN,NaN,NaN,<NA>,29.0,Sunday,7,September
1,3,test drive,26839.7,6434,7004,275.0,Run,Run,0,11712490472,...,NaN,NaN,NaN,NaN,NaN,<NA>,59.2,Saturday,8,June
2,3,pmp bonk,16609.6,4061,4376,30.0,Run,Run,0,11821538649,...,NaN,NaN,NaN,NaN,NaN,<NA>,19.2,Saturday,7,July
3,3,locked and loaded,6457.0,1726,1810,54.0,Run,Run,0,12352617132,...,NaN,NaN,NaN,NaN,NaN,<NA>,23.0,Saturday,15,September
4,3,beep boppin around,17741.5,4502,4706,225.0,Run,Run,0,12155934411,...,NaN,NaN,NaN,NaN,NaN,<NA>,69.0,Thursday,11,August


## Data Exploration

In [4]:
activities.dtypes

resource_state                              Int64
name                                       object
distance                                  float64
moving_time                                 Int64
elapsed_time                                Int64
                                           ...   
similar_activities_frequency_milestone      Int64
elevation_change                          float64
day_of_week                                object
hour                                        Int64
month                                      object
Length: 69, dtype: object

## Data Preprocessing

### Removing Duplicates

In [5]:
activities = activities.drop_duplicates(subset='id', keep='first')

### Drop Empty Columns

In [6]:
# Identify columns that are completely empty
empty_columns = activities.columns[activities.isna().all()]

# Drop columns that are completely empty
activities = activities.drop(columns=empty_columns)

# Print the names of dropped columns (optional)
print("Dropped columns:", empty_columns)

# Print the shape of the new dataframe
print(activities.shape)

# Print the new dataframe
activities.head(5)

Dropped columns: Index(['location_city', 'location_state', 'location_country',
       'similar_activities_effort_count', 'similar_activities_average_speed',
       'similar_activities_min_average_speed',
       'similar_activities_mid_average_speed',
       'similar_activities_max_average_speed', 'similar_activities_pr_rank',
       'similar_activities_frequency_milestone'],
      dtype='object')
(182, 59)


,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,...,device_name,embed_token,athlete_id,gear_primary,gear_name,gear_distance,elevation_change,day_of_week,hour,month
0,3,2024 Erie Marathon,42346.6,9454,9454,101.0,Run,Run,1,12359295567,...,Garmin Forerunner 255S,aefaf206c94f6ead861c43d4335d5ae2eb197247,57248538,0,ASICS METASPEED SKY PARIS,85795,29.0,Sunday,7,September
1,3,test drive,26839.7,6434,7004,275.0,Run,Run,0,11712490472,...,Garmin Forerunner 255S,3d75b535b9ee73d6a85a3e3e191246e15e8bd252,57248538,0,ASICS METASPEED SKY PARIS,85795,59.2,Saturday,8,June
2,3,pmp bonk,16609.6,4061,4376,30.0,Run,Run,0,11821538649,...,Garmin Forerunner 255S,f63d39811f684dea5367d21e8f7a734030da62c1,57248538,0,ASICS METASPEED SKY PARIS,85795,19.2,Saturday,7,July
3,3,locked and loaded,6457.0,1726,1810,54.0,Run,Run,0,12352617132,...,Garmin Forerunner 255S,5598d42fec4009e1986f24015f2d530146c5a6db,57248538,0,ASICS Superblast 2,311088,23.0,Saturday,15,September
4,3,beep boppin around,17741.5,4502,4706,225.0,Run,Run,0,12155934411,...,Garmin Forerunner 255S,25bc1d37865da54746aefd2271c3b7ccc9f71bad,57248538,0,ASICS Superblast 2,311088,69.0,Thursday,11,August


### Checking for missing values

In [7]:
# Get the count of missing values in each column
missing_values = activities.isna().sum()

# Display columns with their respective missing values count
print(missing_values)

resource_state                 0
name                           0
distance                       0
moving_time                    0
elapsed_time                   0
total_elevation_gain           0
type                           0
sport_type                     0
workout_type                   1
id                             0
start_date                     0
start_date_local               0
timezone                       0
achievement_count              0
kudos_count                    0
comment_count                  0
athlete_count                  0
photo_count                    0
trainer                        0
commute                        0
manual                         0
private                        0
visibility                     0
flagged                        0
gear_id                        0
start_latlng                   0
end_latlng                     0
average_speed                  0
max_speed                      0
average_cadence                0
average_wa

In [8]:
activities.drop(['workout_type', 'prefer_perceived_exertion'], axis=1,inplace=True)

### Removing Demo activities

In [9]:
activities = activities[activities['distance']>100]

In [10]:
activities.reset_index(inplace=True)

## Prediciting Calories Burned from Activities Table

In [11]:
activities.columns.values

array(['index', 'resource_state', 'name', 'distance', 'moving_time',
       'elapsed_time', 'total_elevation_gain', 'type', 'sport_type', 'id',
       'start_date', 'start_date_local', 'timezone', 'achievement_count',
       'kudos_count', 'comment_count', 'athlete_count', 'photo_count',
       'trainer', 'commute', 'manual', 'private', 'visibility', 'flagged',
       'gear_id', 'start_latlng', 'end_latlng', 'average_speed',
       'max_speed', 'average_cadence', 'average_watts', 'max_watts',
       'weighted_average_watts', 'kilojoules', 'device_watts',
       'has_heartrate', 'average_heartrate', 'max_heartrate', 'elev_high',
       'elev_low', 'upload_id', 'upload_id_str', 'external_id',
       'pr_count', 'total_photo_count', 'suffer_score', 'calories',
       'perceived_exertion', 'device_name', 'embed_token', 'athlete_id',
       'gear_primary', 'gear_name', 'gear_distance', 'elevation_change',
       'day_of_week', 'hour', 'month'], dtype=object)

In [12]:
# List of selected columns to keep as features
feature_columns = [
    'distance', 'moving_time', 'elapsed_time', 'total_elevation_gain',
    'average_speed', 'max_speed', 'average_cadence', 'max_watts', 'average_watts',
    'kilojoules', 'average_heartrate', 'max_heartrate', 'suffer_score', 'elevation_change'
]

# Keep only the selected columns in your DataFrame
X = activities[feature_columns]
y = activities['calories']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Modeling

### Linear Regression Model

In [14]:
pipeline_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LinearRegression())
])

pipeline_lr.fit(X_train, y_train)
y_pred_lr = pipeline_lr.predict(X_test)

mae_lr = mean_absolute_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)
print("Linear Regression MAE:", mae_lr)
print("Linear Regression R²:", r2_lr)

Linear Regression MAE: 24.8915477154187
Linear Regression R²: 0.9929421851926067


### Random Forest model

In [15]:
pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor(random_state=42))
])

param_grid_rf = {
    'rf__n_estimators': [100, 200, 300],
    'rf__max_depth': [10, 20, 30, None],
    'rf__min_samples_split': [2, 5, 10]
}

grid_search_rf = GridSearchCV(estimator=pipeline_rf, param_grid=param_grid_rf, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=0)
grid_search_rf.fit(X_train, y_train)

best_rf = grid_search_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test)

mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print("Random Forest MAE:", mae_rf)
print("Random Forest R²:", r2_rf)
print("Best Parameters for Random Forest:", grid_search_rf.best_params_)

Random Forest MAE: 33.47718297190356
Random Forest R²: 0.9813937909629401
Best Parameters for Random Forest: {'rf__max_depth': 10, 'rf__min_samples_split': 2, 'rf__n_estimators': 100}


### XGBoost model

In [16]:
pipeline_xgb = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', XGBRegressor(objective='reg:squarederror', random_state=42))
])

param_grid_xgb = {
    'xgb__n_estimators': [100, 200],
    'xgb__max_depth': [3, 6, 10],
    'xgb__learning_rate': [0.01, 0.1, 0.2],
    'xgb__subsample': [0.7, 0.8, 0.9]
}

grid_search_xgb = GridSearchCV(estimator=pipeline_xgb, param_grid=param_grid_xgb, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=0)
grid_search_xgb.fit(X_train, y_train)

best_xgb = grid_search_xgb.best_estimator_
y_pred_xgb = best_xgb.predict(X_test)

mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)
print("XGBoost MAE:", mae_xgb)
print("XGBoost R²:", r2_xgb)
print("Best Parameters for XGBoost:", grid_search_xgb.best_params_)

XGBoost MAE: 29.90284893963788
XGBoost R²: 0.9871418337091263
Best Parameters for XGBoost: {'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 100, 'xgb__subsample': 0.7}


### Best performance model

In [17]:
model_performance = {
    "Linear Regression": {"MAE": mae_lr, "R²": r2_lr},
    "Random Forest": {"MAE": mae_rf, "R²": r2_rf},
    "XGBoost": {"MAE": mae_xgb, "R²": r2_xgb}
}

best_model_mae = min(model_performance, key=lambda x: model_performance[x]["MAE"])
best_model_r2 = max(model_performance, key=lambda x: model_performance[x]["R²"])

print("Best Model based on MAE:")
print(f"{best_model_mae} with MAE = {model_performance[best_model_mae]['MAE']} and R² = {model_performance[best_model_mae]['R²']}")

print("\nBest Model based on R²:")
print(f"{best_model_r2} with R² = {model_performance[best_model_r2]['R²']} and MAE = {model_performance[best_model_r2]['MAE']}")

Best Model based on MAE:
Linear Regression with MAE = 24.8915477154187 and R² = 0.9929421851926067

Best Model based on R²:
Linear Regression with R² = 0.9929421851926067 and MAE = 24.8915477154187
